In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import  mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:

def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    return precision



def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [3]:
data = pd.read_csv('../data/retail_train.csv')
data

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2396799,1613,41655820646,663,16102849,1,2.00,3262,-1.15,1231,95,0.0,0.0
2396800,1001,41655829421,663,13217063,1,1.69,3131,0.00,2231,95,0.0,0.0
2396801,1001,41655829421,663,13217800,1,1.69,3131,0.00,2231,95,0.0,0.0
2396802,1167,41656790510,663,6410462,22451,43.98,3385,-0.65,1059,95,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
Зачем он нужен?  
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
Можно ли удалить этот товар?   
Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

### Ответ к заданию 0  
1. Товар 999999 использовался для подмены на товары которые менее поплярны указаного нами уровня,  внашем случае после первых 5000 товаров. Это мы делали что бы уменьшить матрицу и сделать ее менее разряженой.  
2. При уменьшении матрицы, для целей увелечиения скорости работы с данными, чтобы не терять данные и не пропадали юзеры, которые купили только не популярные товары.
3. В таком виде, товара 99999 в actual нет, но он есть почти в каждом предсказании, следовательно фактически метрика становиться более строгой . Вместо pr@5, получается pr@4. Можно данный товар добавить в actual для , тех кто покупал товары, которые не попали в список. Можно заменять данный товар в рекомендациях следущим .
4. Имеется ввиду не вводить его? Какой смысл сначала преименовать все менее поппулярные товары, а потом их  удалять? Ну как минимум можно работиать со всей матрицей, тогда не нужно будет длеать такой трюк. Можно просто удалить эти транзакции, тогда потреяются юзеры, которые ничего кроме этих товаров не покупали. Но с другой строны в такой модели мы таким юзерам врят ли что то  сможем порекомендовать, поскольтку модель будет учиться на других товарах и не будет видеть возможных связей этих товаров с этими юзерами, то есть предсказание таким юзерам будет больше носить рандомный характер, наверное :) Или имеется ввиду заменить в рекоменлациях на другой товар, как написано в п.3?

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

#### Решение к заданию 1

Конечно если бы на вебинаре или в методичке нам пояснили, что это такое 'сэмплировать товары случайно, но пропорционально какому-либо весу' я бы сэкономил 3 часа времени и поспал не 3 часа , а 6 :) Сначала я долго не мог понять, что это в принципе означает - 'случайно, но пропорционально весу'. Потом изобретал велосипед, еще не факт что правильный  велосипед получился :)

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации   
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    df = items_weights.copy()
    df = df.sort_values('weight',ascending=False)
    r = np.random.randint(n,len(items_weights))
    df= df.iloc[:r]
    recs = np.random.choice(df.iloc[:r,0],size = n,replace=False)
    
    return recs.tolist()

In [6]:
# потестируем  алгоритм
test = pd.DataFrame({'item_id':[1,2,3,4,5,6,7,8,9,10],'weight':[0.3,0.25,0.2,0.15,0.1,0.08,0.07,0.05,0.003,0.001]})

In [7]:
weighted_random_recommendation(test)

[3, 1, 4, 6, 8]

In [8]:
d={}
for i in range(1,11):
    d[i] = 0
    
for i in range(10000):
    a = weighted_random_recommendation(test,n=3)
    for j in a:
        d[j]+=1
d    

{1: 5679,
 2: 5738,
 3: 5701,
 4: 4246,
 5: 3220,
 6: 2331,
 7: 1559,
 8: 1051,
 9: 475,
 10: 0}

вес по сумме продаж

In [9]:
items_weights = data.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['weight'] =  np.log(items_weights.sales_value+2)
items_weights['weight'] =  items_weights['weight'] / items_weights['weight'].sum()
items_weights.drop('sales_value',axis=1,inplace=True)

In [10]:
items_weights.weight.sum()

1.0

In [11]:
items_weights.describe()

,item_id,weight
count,8.905100e+04,89051.000000
mean,5.115772e+06,0.000011
std,5.178973e+06,0.000006
min,2.567100e+04,0.000003
25%,9.665830e+05,0.000007
50%,1.448516e+06,0.000010
75%,9.553042e+06,0.000015
max,1.802456e+07,0.000050


In [12]:
weighted_random_recommendation(items_weights)

[13213293, 1097334, 888570, 9676742, 1101050]

по объему продаж в натуральном выражении

In [13]:
items_weights = data.groupby('item_id')['quantity'].sum().reset_index()
items_weights['weight'] =  np.log(items_weights.quantity+2)
items_weights['weight'] =  items_weights['weight'] / items_weights['weight'].sum()
items_weights.drop('quantity',axis=1,inplace=True)

In [14]:
items_weights.weight.sum()

0.9999999999999999

In [15]:
items_weights.describe()

,item_id,weight
count,8.905100e+04,89051.000000
mean,5.115772e+06,0.000011
std,5.178973e+06,0.000007
min,2.567100e+04,0.000004
25%,9.665830e+05,0.000006
50%,1.448516e+06,0.000008
75%,9.553042e+06,0.000015
max,1.802456e+07,0.000098


In [16]:
weighted_random_recommendation(items_weights)

[1121059, 8203752, 1104377, 12810626, 1139623]

вес по количеству продаж

In [17]:
items_weights = data.groupby('item_id')['sales_value'].count().reset_index()
items_weights['weight'] =  np.log(items_weights.sales_value+2)
items_weights['weight'] =  items_weights['weight'] / items_weights['weight'].sum()
items_weights.drop('sales_value',axis=1,inplace=True)

In [18]:
items_weights.weight.sum()

1.0000000000000002

In [19]:
items_weights.describe()

,item_id,weight
count,8.905100e+04,89051.000000
mean,5.115772e+06,0.000011
std,5.178973e+06,0.000007
min,2.567100e+04,0.000006
25%,9.665830e+05,0.000006
50%,1.448516e+06,0.000009
75%,9.553042e+06,0.000015
max,1.802456e+07,0.000055


In [20]:
weighted_random_recommendation(items_weights)

[915438, 13210037, 9368348, 5574369, 891520]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [21]:
result = pd.read_csv('preds.csv',dtype=np.object)# загрузка predict с семианара
for i in result:
    result[i] = result[i].apply(lambda x: eval(x))
    print(i,type(result[i][0]))


user_id <class 'numpy.int64'>
actual <class 'list'>
random_recommendation <class 'list'>
popular_recommendation <class 'list'>
itemitem <class 'list'>
tfidf <class 'list'>
own_purchases <class 'list'>


In [22]:
result['weighted_random'] = result.user_id.apply(lambda x: weighted_random_recommendation(items_weights))

In [23]:
result.head(3)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,tfidf,own_purchases,weighted_random
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[991268, 965292, 1128665, 904435, 12731714]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[852975, 880343, 1105761, 1000329, 883299]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5567702, 1109014, 1007195, 952698, 924834]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[1128861, 951703, 850807, 1040553, 900072]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1109014, 1022199, 978343, 893651, 901762]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]","[1055646, 7167218, 1048257, 1056509, 929018]"


In [24]:
result_metrics = pd.DataFrame(columns=['precision_at_5'])
for i in result.columns[2:]:
    result_metrics.loc[i,'precision_at_5'] = result.apply(lambda x: precision_at_k(x[i], x['actual'],  5), axis=1).mean()
result_metrics.sort_values('precision_at_5',ascending=False)   

,precision_at_5
own_purchases,0.179693
popular_recommendation,0.15524
tfidf,0.138981
itemitem,0.136925
random_recommendation,0.00519099
weighted_random,0.00235064


Наврное для адекватного сравнения будет справделиво если weighted_random и random_recommendation будет так же браться из списка 5000 самых популярных товров. Поскольку для расчета остальных рекомендаций мы брали именно их

In [25]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [26]:

popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999


C:\Users\lqw\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [27]:
result.random_recommendation = result.user_id.apply(lambda x:random_recommendation(data_train.item_id.unique()))

In [28]:
items_weights = data_train.groupby('item_id')['sales_value'].count().reset_index()
items_weights['weight'] =  np.log(items_weights.sales_value+2)
items_weights['weight'] =  items_weights['weight'] / items_weights['weight'].sum()
items_weights.drop('sales_value',axis=1,inplace=True)

In [29]:
result['weighted_random'] = result.user_id.apply(lambda x: weighted_random_recommendation(items_weights))

In [30]:

for i in result.columns[2:]:
    result_metrics.loc[i,'precision_at_5_1'] = result.apply(lambda x: precision_at_k(x[i], x['actual'],  5), axis=1).mean()
result_metrics.sort_values('precision_at_5_1',ascending=False)   

,precision_at_5,precision_at_5_1
own_purchases,0.179693,0.179693
popular_recommendation,0.15524,0.155240
tfidf,0.138981,0.138981
itemitem,0.136925,0.136925
weighted_random,0.00235064,0.014300
random_recommendation,0.00519099,0.005387


### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [31]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()



In [32]:
# перенумеруем пользователей и товары
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [33]:
%%time
for i in range(2,11):
    model = ItemItemRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

    name = 'itemitem_k_'+str(i)
    result[name] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])

Wall time: 3.87 s


In [34]:
%%time
for i in range(2,11):
    model = CosineRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

    name = 'cosine_k_'+str(i)
    result[name] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])

Wall time: 4.46 s


In [35]:
%%time
for i in range(2,11):
    model = CosineRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

    name = 'cosine_k_'+str(i)
    result[name] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])

Wall time: 4.17 s


In [36]:
%%time
for i in range(2,11):
    model = TFIDFRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)

    name = 'tfidf_k_'+str(i)
    result[name] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])

Wall time: 4.39 s


In [37]:
result.shape,sparse_user_item.shape

((2042, 35), (2499, 5001))

In [38]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,tfidf,own_purchases,weighted_random,itemitem_k_2,itemitem_k_3,...,cosine_k_10,tfidf_k_2,tfidf_k_3,tfidf_k_4,tfidf_k_5,tfidf_k_6,tfidf_k_7,tfidf_k_8,tfidf_k_9,tfidf_k_10
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[849809, 937928, 1139830, 853178, 998984]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[891542, 823758, 981874, 976998, 870128]","[999999, 1082185, 995242, 1029743, 840361]","[999999, 1082185, 981760, 995242, 1029743]",...,"[1082185, 999999, 981760, 1127831, 961554]","[1082185, 999999, 961554, 840361, 979707]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 981760, 1127831, 999999, 961554]","[1082185, 981760, 1127831, 961554, 999999]","[1082185, 981760, 1127831, 961554, 999999]","[1082185, 981760, 1127831, 999999, 961554]","[1082185, 981760, 1127831, 999999, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1119289, 9834988, 1055412, 8293447, 1129321]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[855557, 1128900, 993466, 883370, 1115228]","[999999, 1082185, 1098066, 6534178, 826249]","[999999, 1082185, 981760, 1098066, 6534178]",...,"[1082185, 1098066, 981760, 999999, 826249]","[1082185, 1098066, 999999, 883404, 826249]","[1082185, 1098066, 999999, 826249, 883404]","[1082185, 1098066, 981760, 826249, 999999]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 981760, 826249, 999999]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 981760, 1098066, 826249, 1044078]","[1082185, 1098066, 981760, 826249, 1044078]"


In [39]:
%%time
try:
    result_metrics = result_metrics.drop('precision_at_5',axis=1)
except:
    pass
    
for i in result.columns[2:]:
    result_metrics.loc[i,'precision_at_5_1'] = result.apply(lambda x: precision_at_k(x[i], x['actual'],  5), axis=1).mean()
result_metrics.sort_values('precision_at_5_1',ascending=False)   

Wall time: 2.5 s


,precision_at_5_1
itemitem_k_2,0.192009
itemitem_k_3,0.186092
own_purchases,0.179693
tfidf_k_2,0.155950
popular_recommendation,0.155240
itemitem_k_10,0.150930
cosine_k_2,0.149780
itemitem_k_9,0.148482
itemitem_k_8,0.147209
tfidf_k_3,0.146425


ансамблирование - опять таки предлагается снова , что то сделать, что не затрагивали на вебинаре. Можно было бы об этом сказать. Или хотя бы поянить основной принцип.
Я еще представляю классификация и регрессия , там mean, max, min можно сделать. А как из двух списков по 5 позиций сделать один список из пяти позиций? Я не знаю как правилльно, могу только предполагать и тратить опять время на решение ребусов. Хотя мне нужны знания и опыт.

### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.

#### Ответ 4

я рискну предположить, что  
ru - это средний ретинг  юзера u по всем товарам,  
мю это средний рейтинг всех юзеров по всем товарам  
rv - средний вес v юзероа про всем товарам

Если често, вообще не понимаю сути этой формулы


Считаем

In [61]:
quantity_user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )
# перенумеруем пользователей и товары
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))


quantity_user_item_matrix = quantity_user_item_matrix.to_numpy()

In [62]:
mu = quantity_user_item_matrix.mean()

In [63]:
norma = np.sqrt((quantity_user_item_matrix**2).sum(axis=1))

In [64]:
def knn(user,k=5):
    skal = quantity_user_item_matrix@user
    norma_user = np.sqrt((user**2).sum())
    cos = skal/(norma*norma_user)
    return cos.argsort()[-1:-k-1:-1] 

In [65]:
def ru(user):
    return user.mean()

In [66]:
def rv(knn):
    return quantity_user_item_matrix[knn].mean()

In [67]:
def rvi(knn):
    res = quantity_user_item_matrix[knn]
    return res.mean(axis = 0)

In [68]:
def rui(user,k=5):
    ru_ = ru(user)
    knn_=knn(user,k)
    rvi_ = rvi(knn_)
    rv_ = rv(knn_)
    return mu + ru_ + rvi_ - rv_ - mu/k


In [69]:
n_users, n_items = quantity_user_item_matrix.shape

In [70]:
ratio_user_item_matrix = np.zeros([n_users,n_items])

In [71]:
%%time
for  i,current_user in enumerate(quantity_user_item_matrix):
    ratio_user_item_matrix[i] = rui(current_user)
    
    
    

Wall time: 2min 25s


In [72]:
ratio_user_item_matrix

array([[0.23447482, 1.23447482, 0.23447482, ..., 0.23447482, 0.23447482,
        0.23447482],
       [0.06350901, 0.06350901, 0.06350901, ..., 0.06350901, 0.26350901,
        0.06350901],
       [0.00999971, 0.00999971, 0.00999971, ..., 0.00999971, 0.20999971,
        0.00999971],
       ...,
       [0.45351101, 0.05351101, 0.05351101, ..., 0.05351101, 0.25351101,
        0.05351101],
       [0.00999971, 0.00999971, 0.00999971, ..., 0.00999971, 0.20999971,
        0.00999971],
       [0.15641043, 0.15641043, 0.15641043, ..., 0.15641043, 0.35641043,
        0.15641043]])

In [73]:
ratio_user_item_matrix.shape

(2499, 5001)

In [74]:
recomends = np.zeros([n_users,5])
for k,current_user in enumerate(ratio_user_item_matrix):
    ind = np.argsort(current_user)[-1:-6:-1]
    for i,j in enumerate(ind):
        recomends[k,i]=id_to_itemid[j]
recomends

array([[ 999999., 1082185.,  995242.,  856942.,  940947.],
       [ 999999., 1106523., 1082185., 1133018., 5569230.],
       [ 999999., 1106523., 1092026.,  844165.,  951590.],
       ...,
       [ 999999., 1082185., 1126899., 6534178., 1070820.],
       [ 999999., 1070820., 1082185.,  995785.,  862349.],
       [ 999999., 1082185., 6534178., 1126899., 1058997.]])

In [75]:
recomends.shape

(2499, 5)

In [76]:
predict_res = []
for i in result.user_id:

    predict_res.append(precision_at_k(recomends[userid_to_id[i]], result[result.user_id==i]['actual'],  5))
predict_res = np.array(predict_res)
predict_res.mean()


0.0

In [77]:
predict_res.sum()

0.0

Результата нет, что то я не так понял....